<a href="https://colab.research.google.com/github/dominikaandrzejewska/natural_language_processing/blob/master/NLP_challenge_day3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install gensim
#!pip install pyLDAvis

In [0]:
import gzip
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import Word2Vec, LdaMulticore

from gensim.corpora import Dictionary
import numpy as np
import pyLDAvis
from pyLDAvis import gensim

In [0]:
def preprocessing(sentence):
  return [word for word in simple_preprocess(sentence) if word not in STOPWORDS] 

def read_sentences(filename):
  file = open(filename, 'rb')
  for line in file:
    yield preprocessing(line)

%time sentences = list(read_sentences('data/reviews_data.txt'))

CPU times: user 49.7 s, sys: 1.21 s, total: 50.9 s
Wall time: 51.3 s


In [0]:
len(sentences)

255404

## Model

In [0]:
model = Word2Vec(sentences, size=100, window=5, min_count=2)
#model = Word2Vec.load('models/hotel_model_size300_window5_mincount2.model.wv.vectors') <-- loading a previously prepared model

## Reprezentacja Wektorowa

In [0]:
model.wv.most_similar('good')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('decent', 0.8320484161376953),
 ('great', 0.8161169290542603),
 ('excellent', 0.7963471412658691),
 ('ok', 0.6623174548149109),
 ('fair', 0.6367666125297546),
 ('average', 0.6257697343826294),
 ('reasonable', 0.6215258836746216),
 ('terrific', 0.6071202754974365),
 ('fantastic', 0.6027746200561523),
 ('nice', 0.5996159911155701)]

In [0]:
 model.wv.most_similar(positive = ['bad','terrible'], negative=['good'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('horrible', 0.8536473512649536),
 ('awful', 0.8087409138679504),
 ('dreadful', 0.7382410764694214),
 ('appalling', 0.690683901309967),
 ('horrendous', 0.686642050743103),
 ('shocking', 0.6692348122596741),
 ('horrid', 0.6668890714645386),
 ('disgusted', 0.6514927744865417),
 ('miserable', 0.6431301832199097),
 ('nasty', 0.6394613981246948)]

In [0]:
model.wv.most_similar('london')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('nyc', 0.8272808194160461),
 ('sf', 0.814702033996582),
 ('manhattan', 0.7982243895530701),
 ('ny', 0.774950385093689),
 ('dubai', 0.765128493309021),
 ('beijing', 0.7568119764328003),
 ('chicago', 0.7553144693374634),
 ('montreal', 0.7435177564620972),
 ('city', 0.6903071999549866),
 ('delhi', 0.6837425231933594)]

In [0]:
model.wv.most_similar('hotel')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('property', 0.7270717620849609),
 ('place', 0.6298233866691589),
 ('hotels', 0.6095315217971802),
 ('resort', 0.5777890682220459),
 ('establishment', 0.5534394979476929),
 ('accommodation', 0.5304919481277466),
 ('accomodation', 0.5232415199279785),
 ('accomodations', 0.5170595049858093),
 ('location', 0.5047421455383301),
 ('travelodge', 0.49539434909820557)]

## Topic Modelling - LDA

In [0]:
sentences_light = np.random.permutation(sentences)

In [0]:
sentences_light = sentences_light[:1000]

In [0]:
%time dictionary = Dictionary(sentences_light)

CPU times: user 98.6 ms, sys: 8 µs, total: 98.6 ms
Wall time: 99.1 ms


In [0]:
%time bow_corpus = [dictionary.doc2bow(sent) for sent in sentences_light]

CPU times: user 72.6 ms, sys: 3.01 ms, total: 75.6 ms
Wall time: 77.1 ms


In [0]:
#%time lda_model = LdaMulticore(bow_corpus, id2word=dictionary, num_topics=100, passes=20, workers=8)

In [0]:
for idx, topic in lda_model.print_topics(-1):
  print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.036*"hotel" + 0.023*"room" + 0.015*"like" + 0.012*"stay" + 0.010*"great" + 0.009*"night" + 0.008*"service" + 0.007*"floor" + 0.007*"park" + 0.007*"walking"
Topic: 1 
Words: 0.015*"parking" + 0.012*"park" + 0.012*"hotel" + 0.009*"stay" + 0.008*"court" + 0.008*"earls" + 0.008*"walk" + 0.008*"quot" + 0.007*"experience" + 0.007*"room"
Topic: 2 
Words: 0.010*"great" + 0.009*"rooms" + 0.009*"room" + 0.008*"stay" + 0.008*"nice" + 0.007*"staff" + 0.006*"quot" + 0.006*"hotel" + 0.006*"like" + 0.006*"know"
Topic: 3 
Words: 0.034*"hotel" + 0.012*"room" + 0.010*"rooms" + 0.009*"good" + 0.009*"helpful" + 0.008*"service" + 0.008*"got" + 0.008*"restaurant" + 0.008*"stayed" + 0.007*"breakfast"
Topic: 4 
Words: 0.032*"room" + 0.028*"great" + 0.019*"hotel" + 0.016*"quot" + 0.012*"suite" + 0.010*"booked" + 0.010*"good" + 0.009*"price" + 0.008*"coffee" + 0.008*"concierge"
Topic: 5 
Words: 0.021*"room" + 0.017*"hotel" + 0.017*"good" + 0.012*"check" + 0.011*"floor" + 0.011*"shower" + 0.01

## Topics Visualisation

In [0]:
lda_vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
pyLDAvis.display(lda_vis)

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
